In [1]:
import keras
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [3]:
import os
def plot_imgs(directory, top=10):
    all_item_dirs = os.listdir(directory)
    item_files = [os.path.join(directory, file) for file in all_item_dirs][:5]
  
    plt.figure(figsize=(20, 20))
  
    for i, img_path in enumerate(item_files):
        plt.subplot(10, 10, i+1)
    
        img = plt.imread(img_path)
        plt.tight_layout()         
        plt.imshow(img, cmap='gray') 

In [5]:
batch_size = 32
train_datagen = ImageDataGenerator(horizontal_flip = True, 
                                  rescale = 1./255, 
                                  zoom_range = 0.2, 
                                  validation_split = 0.1)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
train_data_path = 'dataset/train'
test_data_path = 'dataset/test'

train_set = train_datagen.flow_from_directory(train_data_path, target_size = (64,64),
                                              batch_size = batch_size, 
                                              color_mode = 'grayscale',
                                              class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_data_path, target_size = (64,64),
                                              batch_size = batch_size, 
                                              color_mode = 'grayscale',
                                              class_mode = 'categorical')

Found 3400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [7]:
classes = 2

model = Sequential()
model.add(Conv2D(32, (3,3), padding = 'same', input_shape = (64,64,1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))

model.add(Dense(classes, activation = 'softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 8, 8, 128)         0

In [8]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam' , metrics = ['accuracy'])

In [9]:
model_path="drowsy_detector.h5"

checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, 
                              save_best_only=True, mode='max')

callbacks_list = [checkpoint]

In [10]:
num_epochs = 20
training_steps=train_set.n//train_set.batch_size
validation_steps =test_set.n//test_set.batch_size

In [11]:
history = model.fit_generator(train_set, epochs=num_epochs, steps_per_epoch=training_steps,validation_data=test_set,
                    validation_steps=validation_steps, callbacks = callbacks_list)

C:\Users\DELL\AppData\Local\Temp\ipykernel_12076\2180583923.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_set, epochs=num_epochs, steps_per_epoch=training_steps,validation_data=test_set,


Epoch 1/20
106/106 [==============================] - ETA: 0s - loss: 0.3521 - accuracy: 0.8275
Epoch 1: val_accuracy improved from -inf to 0.97569, saving model to drowsy_detector.h5
106/106 [==============================] - 36s 316ms/step - loss: 0.3521 - accuracy: 0.8275 - val_loss: 0.0728 - val_accuracy: 0.9757
Epoch 2/20


C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


106/106 [==============================] - ETA: 0s - loss: 0.0834 - accuracy: 0.9715
Epoch 2: val_accuracy improved from 0.97569 to 0.97917, saving model to drowsy_detector.h5
106/106 [==============================] - 32s 298ms/step - loss: 0.0834 - accuracy: 0.9715 - val_loss: 0.0620 - val_accuracy: 0.9792
Epoch 3/20
106/106 [==============================] - ETA: 0s - loss: 0.0616 - accuracy: 0.9804
Epoch 3: val_accuracy improved from 0.97917 to 0.99653, saving model to drowsy_detector.h5
106/106 [==============================] - 32s 299ms/step - loss: 0.0616 - accuracy: 0.9804 - val_loss: 0.0125 - val_accuracy: 0.9965
Epoch 4/20
106/106 [==============================] - ETA: 0s - loss: 0.0371 - accuracy: 0.9875
Epoch 4: val_accuracy improved from 0.99653 to 1.00000, saving model to drowsy_detector.h5
106/106 [==============================] - 31s 296ms/step - loss: 0.0371 - accuracy: 0.9875 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 5/20
106/106 [============================

In [14]:
model_json = model.to_json()
with open("drowsy.json","w") as json_file:
    json_file.write(model_json)